# HTML Parser Code-Beautiful Soup -Data Preprocessing code

In [26]:
import requests
from bs4 import BeautifulSoup
import json
import psycopg2
import re
from urllib.parse import urlparse, urlunparse

class General:
    def __init__(self, main_url, start_page, end_page, final_file):
        self.main_url = main_url
        self.start_page = start_page
        self.end_page = end_page
        self.final_file = final_file

    def write_to_file(self, data):
        with open(self.final_file, 'a') as file:
            file.write(data + "\n")

    def clean_text(self, text):
        text = re.sub('<[^<]+?>', '', text)  # Remove HTML tags
        text = re.sub('\s+', ' ', text)  # Replace multiple spaces with a single space
        text = text.strip()  # Remove leading and trailing spaces
        return text

    def normalize_url(self, url):
        parsed = urlparse(url)
        normalized = parsed._replace(fragment="", query="").geturl().rstrip('/')
        return normalized

class AnandTech(General):
    def __init__(self, main_url, start_page, end_page, final_file, db_params):
        super().__init__(main_url, start_page, end_page, final_file)
        self.db_params = db_params
        self.conn = psycopg2.connect(**db_params)
        self.cur = self.conn.cursor()
        self.create_table()

    def create_table(self):
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS News_articles (
            id SERIAL PRIMARY KEY,
            title TEXT,
            author TEXT,
            date_published TEXT,
            url TEXT UNIQUE,
            content TEXT
        );
        '''
        self.cur.execute(create_table_query)
        self.conn.commit()

    def article_exists(self, url):
        normalized_url = self.normalize_url(url)
        query = "SELECT EXISTS(SELECT 1 FROM News_articles WHERE url = %s)"
        self.cur.execute(query, (normalized_url,))
        return self.cur.fetchone()[0]

    def insert_article(self, title, author, date_published, url, content):
        normalized_url = self.normalize_url(url)
        if self.article_exists(normalized_url):
            print(f"Article already exists: {normalized_url}")
            return
        
        if title and author and content and url:
            insert_query = '''
            INSERT INTO News_articles (title, author, date_published, url, content) 
            VALUES (%s, %s, %s, %s, %s) ON CONFLICT (url) DO NOTHING;
            '''
            self.cur.execute(insert_query, (title, author, date_published, normalized_url, self.clean_text(content)))
            self.conn.commit()

    def parse(self):
        article_count = 0

        for page in range(self.start_page, self.end_page + 1):
            url = f"{self.main_url}&CurrentPage={page}&q=arm&sort=date"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            articles = soup.find_all('div', class_='cont_box1')

            for article in articles:
                article_count += 1
                link_tag = article.find('a')
                if link_tag:
                    article_url = self.normalize_url('https://www.anandtech.com' + link_tag['href'])
                    self.process_article(article_url)

        print(f"Total number of articles found: {article_count}")

    def process_article(self, article_url):
        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        title_tag = soup.find('title')
        title = self.clean_text(title_tag.get_text(strip=True)) if title_tag else 'Title not found'

        content_div = soup.find('div', class_='articleContent')
        content = self.clean_text(content_div.get_text(strip=True)) if content_div else 'Content not found'

        script_json_ld = soup.find('script', type='application/ld+json')
        if script_json_ld:
            json_str = re.sub(r'[\n\t\r]', '', script_json_ld.string)
            try:
                data = json.loads(json_str)
                if isinstance(data, dict):
                    data_list = [data]
                elif isinstance(data, list):
                    data_list = data
                else:
                    data_list = []
                
                for item in data_list:
                    author_info = item.get('author', {})
                    if isinstance(author_info, dict):
                        author = author_info.get('name', 'Author not found')
                    elif isinstance(author_info, list) and author_info:
                        author = author_info[0].get('name', 'Author not found') if isinstance(author_info[0], dict) else 'Author not found'
                    else:
                        author = 'Author not found'
                    
                    date_published = item.get('datePublished', 'Date not found')
                    self.print_article(title, author, date_published, article_url, content)
                    self.insert_article(title, author, date_published, article_url, content)
                    break

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from {article_url}: {e}")
        else:
            print("No JSON-LD script found in the article.")

    def print_article(self, title, author, date_published, url, content):
        print(f"Title: {title}\nAuthor: {author}\nDate Published: {date_published}\nURL: {url}\nContent: {content}\n")

# Database Parameters
db_params = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': '1234',
    'host': 'localhost'
}

# Main URL adjusted to include base search query 'q=arm' and sort parameter
main_url = 'https://www.anandtech.com/SearchResults?q=Arm'
parser = AnandTech(main_url, 1, 30, 'AnandTech_articles.txt', db_params)
parser.parse()


Title: Amazon Unveils Graviton4: A 96-Core ARM CPU with 536.7 GBps Memory Bandwidth
Author: Anton Shilov
Date Published: 2023-11-29T21:30:00Z
URL: https://www.anandtech.com/show/21172/amazon-unveils-graviton4-a-96core-cpu-with-5367-gbs-memory-bandwidth
Content: Nowadays many cloud service providers design their own silicon, but Amazon Web Services (AWS) started to do this ahead of its rivals and by now its Annapurna Labs subsidiary develop processors that can well compete with those from AMD and Intel. This week AWS introduced its Graviton4 SoC, a 96-core ARM-based chip that promises to challenge renowned CPU designers and offer unprecedented performance to AWS clients."By focusing our chip designs on real workloads that matter to customers, we are able to deliver the most advanced cloud infrastructure to them," said David Brown, vice president of Compute and Networking at AWS. "Graviton4 marks the fourth generation we have delivered in just five years, and is the most powerful and ene

Title: Qualcomm Swaps Out Arm for RISC-V for Next-Gen Google Wear OS Devices
Author: Anton Shilov
Date Published: 2023-10-18T23:00:00Z
URL: https://www.anandtech.com/show/21098/qualcomm-swaps-out-arm-for-risc-v-for-next-wear-soc
Content: As part of a broad collaborative agreement with Google, Qualcomm this week said that that it will be adopting the RISC-V instruction set architecture (ISA) for a future Snapdragon Wear platform. Working together, the two companies will be bootstrapping a RISC-V ecosystem for Wear OS devices, with Qualcomm providing the hardware while Google expands its wearables OS and associated ecosystem of tools to support the new processor architecture.Qualcomm's Wear processors have been the de facto chip of choice for Wear OS devices since the launch of Google's wearables platform almost a decade ago, with Qualcomm employing multiple generations of Arm CPU designs. This makes Qualcomm's decision to develop a RISC-V wearables SoC especially significant, as it not 

Title: Arm Unveils 2023 Mobile CPU Core Designs: Cortex-X4, A720, and A520 - the Armv9.2 Family
Author: Gavin Bonshor
Date Published: 2023-05-29T00:30:00Z
URL: https://www.anandtech.com/show/18871/arm-unveils-armv92-mobile-architecture-cortex-x4-a720-and-a520-64bit-exclusive
Content: Throughout the world, if there's one universal constant in the smartphone and mobile device market, it's Arm. Whether it's mobile chip makers basing their SoCs on Arm's fully synthesized CPU cores, or just relying on the Arm ISA and designing their own chips, at the end of the day, Arm underlies virtually all of it. That kind of market saturation and relevance is a testament to all of the hard work that Arm has done in the last few decades getting to this point, but it's also a grave responsibility – for most mobile SoCs, their performance only moves forward as quickly as Arm's own CPU core designs and associated IP do.Consequently, we've seen Arm settle into a yearly cadence for their client IP, and this 

Title: Arm Announces Neoverse V2 and E2: The Next Generation of Arm Server CPU Cores
Author: Ryan Smith
Date Published: 2022-09-15T12:00:00Z
URL: https://www.anandtech.com/show/17575/arm-announces-neoverse-v2-and-e2-the-next-generation-of-arm-server-cpu-cores
Content: Just under four years ago, Arm announced their Neoverse family of infrastructure CPU designs. Deciding to double-down on the server and edge computing markets by designing Arm CPU cores specifically for those markets – and not just recycling the consumer-focused Cortex-A designs – Arm set about tackling the infrastructure market in a far more aggressive manner. Those efforts, in turn, have increasingly paid off handsomely for Arm and its partners, whom thanks to the likes of products like Amazon’s Graviton and Ampere Altra CPUs have at long last been able take a meaningful piece of the server CPU market.But as Arm CPUs finally achieve the market penetration that eluded them in the previous decade, Arm needs to make sure i

Title: NVIDIA-Arm Acquisition Officially Nixed, SoftBank to IPO Arm Instead
Author: Ryan Smith
Date Published: 2022-02-08T11:00:00Z
URL: https://www.anandtech.com/show/17245/nvidiaarm-acquisition-officially-nixed-softbank-to-ipo-arm-instead
Content: NVIDIA’s year-and-a-half long effort to acquire Arm has come to an end this morning, as NVIDIA and Arm owner SoftBank have announced that the two companies are officially calling off the acquisition. Citing the current lack of regulatory approval of the deal and the multiple investigations that have been opened up into it, NVIDIA and SoftBank are giving up on their acquisition efforts, as the two firms no longer believe it will be possible to receive the necessary regulatory approvals needed to close the deal. In lieu of being able to sell Arm to NVIDIA (or seemingly anyone else), SoftBank is announcing that they will instead be taking Arm public.First announced back in September of 2020,SoftBank and NVIDIA unveiled what was at the time a $

...

Title: Fireside Interview on 5G with Cristano Amon and Alex Katouzian, Qualcomm
Author: Dr. Ian Cutress
Date Published: 2019-12-02T13:00:00Z
URL: https://www.anandtech.com/show/15187/fireside-interview-on-5g-with-cristano-amon-and-alex-katouzian-qualcomm
Content: A little while back, ahead of Qualcomm’s upcoming Tech Summit, we spoke to President of Qualcomm Cristano Amon and SVP/GM of Qualcomm’s Mobile Technologies Alex Katouzian. Qualcomm’s march in the high-end smartphone space during 2019 has been impressive, capturing all the flagship devices from companies that don’t build their own silicon. 2019 has been a key arena for 5G as well, with over 150 5G smartphones coming to market all on Qualcomm solutions. Then there’s the little matter of Windows on Snapdragon, hopefully becoming a tour-de-force with the 8cx chipset.We quizzed Cristiano and Alex on all three sides of business and how it relates to Qualcomm’s partnerships, strategy, and future. The discussion provided a deeper insi

Title: Junk or Treasure? We Tested Amazon’s Cheapest Black Friday Desktop PC, only $60!
Author: Dr. Ian Cutress
Date Published: 2019-12-01T14:15:00Z
URL: https://www.anandtech.com/show/15186/amazons-cheapest-black-friday-desktop-pc
Content: Some Black Friday deals are wild. A store might offer only a couple of units of a particular TV, discounted by 66%. There might be a few pieces of a flagship smartphone at your local electronics store at half price. These are designed to entice customers through the door, and if you’re brave enough, ensure the cold for up to 12 hours to get that bargain of the year. But one of the key observations about looking at Amazon’s Computing and Components section every Black Friday, particularly this year, is that most of the discounts are for complete trash. After the headline external storage discounts, it’s just page after page of USB cables and smartphone holders. But one thing did catch my eye: an entire PC, for only £57 / $61! How can an entire x86 de

Title: MediaTek Announces Dimensity 1000 SoC: Back To The High-End With 5G
Author: Andrei Frumusanu
Date Published: 2019-11-26T05:30:00Z
URL: https://www.anandtech.com/show/15155/mediatek-announces-dimensity-1000-soc-back-to-the-highend-with-5g
Content: Today MediaTek is announcing the new Dimensity 1000 SoC – the company’s new 5G flagship SoC for 2019 and early 2020. The announcement really isn’t too much of a surprising new reveal, as the “5G SoC” wasannounced some time ago in Maywith the company describing its 5G capabilities as well as the inclusion of the new Cortex-A77 and Mali-G77 IPs. What MediaTek does today however is to officially name the new chipset, and go into a few more details on the specifications.First of all, the naming of the chipset is part of rebrand for the company’s SoC line-up going forward. The “Dimensity” name is meant to usher in a new generation and differentiate the new 5G SoC from the previous 4G Helio line-up.“We chose the name Dimensity to highlight ho

Title: New NVIDIA GPU Variant Found at Supercomputing 2019: Tesla V100S
Author: Dr. Ian Cutress
Date Published: 2019-11-22T20:00:00Z
URL: https://www.anandtech.com/show/15146/new-nvidia-gpu-variant-at-supercomputing-2019
Content: NVIDIA announced a number of things at Supercomputing, such as an Arm server referece design. Despite the show being the major hub event for high-performance computing/supercomputers, it isn’t often the location where NVIDIA launches a GPU. Nonetheless we saw a new model of NVIDIA’s high performance Tesla V100 at multiple booths at Supercomputing.The new GPU we saw was called the V100S (or V100s). Firstly, the name: I didn’t realise it was new/unannounced until it was pointed out to me. The way it was written on a few of the billboards looks like it is just referring to ‘multiple V100 units’, but a couple of companies confirmed to be that it is a new product. For these vendors, they were actually told before the show that NVIDIA was planning to announce it the

Title: The Intel Core i9-9900KS Review: The 5 GHz Consumer Special
Author: Dr. Ian Cutress
Date Published: 2019-10-31T14:45:00Z
URL: https://www.anandtech.com/show/14979/the-intel-core-i9-9900ks-review
Content: Intellikes 5.0 GHz processors. The one area where it claims a clear advantage over AMD is in its ability to drive the frequency of its popular 14nm process. Earlier this week,we reviewed the Core i9-9990XE, which is a rare auction only CPU but with 14 cores at 5.0 GHz, built for the high-end desktop and high frequency trading market. Today we are looking at its smaller sibling, the Core i9-9900KS, built in numbers for the consumer market: eight cores at 5.0 GHz. But you’ll have to be quick, as Intel isn’t keeping this one around forever.The Battle of the BitsEvery time a new processor comes to market, several questions get asked: how many cores, how fast, how much power? We’ve come through generations of promises of many GHz and many cores for little power, but right now we have

# "RogerKam/roberta_fine_tuned_sentiment_newsmtsc"

In [2]:
import psycopg2
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.nn.functional import softmax

# Database connection parameters
db_host = "localhost"
db_name = "postgres"
db_user = "postgres"
db_password = "1234"

# Connect to your database
conn = psycopg2.connect(host=db_host, dbname=db_name, user=db_user, password=db_password)
cursor = conn.cursor()

# Modify the table to include new columns
cursor.execute('''
    CREATE TABLE IF NOT EXISTS article_sentiments_RogerKam (
        article_id INTEGER PRIMARY KEY REFERENCES news_articles(id),
        sentiment TEXT,
        url TEXT,
        author TEXT,
        date_published DATE,
        title TEXT
    );
''')
conn.commit()

# Update SQL query to fetch additional fields
sql_query = "SELECT id, content, url, author, date_published, title FROM news_articles"

# Initialize model and tokenizer
model_id = "RogerKam/roberta_fine_tuned_sentiment_newsmtsc"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)
max_token_limit = tokenizer.model_max_length
priotity_label = 'negative'

# Your existing sentiment_analysis function remains the same
def sentiment_analysis(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    predictions = softmax(outputs.logits, dim=1)
    # Assuming the sentiment labels are ordered as ["negative", "neutral", "positive"]
    sentiment_labels = ['negative', 'neutral', 'positive']
    sentiment_score, sentiment_index = torch.max(predictions, dim=1)
    max_score_index = torch.argmax(predictions[:, 1:]) + 1  # Exclude "negative" from max score calculation
    if sentiment_index == 0:
        sentiment = priotity_label
    else:
        sentiment = sentiment_labels[max_score_index.item()]
    return sentiment
 
 
 #Function to find the maximum sentiment label for a combined sentence
def max_sentiment_label(sentiment_labels):
    max_sentiment = max(sentiment_labels, key=sentiment_labels.count)
    return max_sentiment
# Fetch data, perform sentiment analysis for each sentence, and store results
try:
    cursor.execute(sql_query)
    records = cursor.fetchall()
    for record in records:
        article_id, content, url, author, date_published, title = record
        sentences = [content[i:i + max_token_limit] for i in range(0, len(content), max_token_limit)]
        
        results = []
        for sentence in sentences:
            sentiment = sentiment_analysis(sentence)
            if sentiment == priotity_label:
                results = [sentiment]
                break
            else:
                results.append(sentiment)
                
        combined_sentiment = priotity_label if priotity_label in results else max_sentiment_label(results)

        # Update insert query to include new fields
        insert_query = """
        INSERT INTO article_sentiments_RogerKam (article_id, sentiment, url, author, date_published, title)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (article_id) DO UPDATE SET
        sentiment = EXCLUDED.sentiment,
        url = EXCLUDED.url,
        author = EXCLUDED.author,
        date_published = EXCLUDED.date_published,
        title = EXCLUDED.title
        """
        cursor.execute(insert_query, (article_id, combined_sentiment, url, author, date_published, title))
        conn.commit()

except Exception as e:
    print(f"Database error: {e}")
finally:
    cursor.close()
    conn.close()
